# MoeaBench: Benchmark Gallery
This notebook provides a visual inspection of the **optimal Pareto fronts** for all implemented benchmark problems in `MoeaBench` v0.7.5. 

It leverages the `topo_shape` perspective, which automatically detects the dimensionality and provides interactive 3D visualizations (via Plotly) for objective spaces where $M=3$.

In [ ]:
import os
import sys
import numpy as np

# Ensure project root is in path
BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)

import MoeaBench as mb

## 1. DTLZ Family (Standard and Scalable)
Iterating through DTLZ1 to DTLZ9 with $M=3$.

In [ ]:
dtlz_names = [f"DTLZ{i}" for i in range(1, 10)]

for name in dtlz_names:
    print(f"Rendering {name}...")
    mop_cls = getattr(mb.mops, name)
    mop = mop_cls(M=3)
    
    # optimal() returns a Population object containing the analytical Pareto Set/Front
    pf = mop.optimal(n_points=500)
    
    # topo_shape is the semantic perspective for geometry
    mb.view.topo_shape(pf, title=f"{name} (M=3) Pareto Front")

## 2. DPF Family (Degenerate Pareto Fronts)
Iterating through DPF1 to DPF5 with $M=3$.

In [ ]:
dpf_names = [f"DPF{i}" for i in range(1, 6)]

for name in dpf_names:
    print(f"Rendering {name}...")
    mop_cls = getattr(mb.mops, name)
    # DPFs require D parameter (reference dimensionality)
    mop = mop_cls(M=3, D=2)
    
    pf = mop.optimal(n_points=500)
    
    mb.view.topo_shape(pf, title=f"{name} (M=3) Pareto Front")